In [8]:
## DATA MANIPULATION
import numpy as np, pandas as pd
import pickle

from collections import defaultdict

## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

## CLASSIFICATION
from sklearn.base import clone
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### Load preprocessed data

In [9]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
labels = train.columns[2:]

### TF-IDF weighted GloVe vectorizer
- TF-IDF down-weighs frequent terms and up-weighs rare terms, which helps identify indicators for toxic comments (i.e., they're less common)

In [10]:
## BUILD GLOVE EMBEDDINGS DICTIONARY
embeddings_dict = dict()
f = open(r'../data/glove.6B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    embeddings_dict[word] = vec
f.close()
print('Extracted {} word vectors'.format(len(embeddings_dict)))

Extracted 400000 word vectors


In [11]:
## SKLEARN COMPATIBLE GLOVE VECTORIZER TRANSFORMER 
class tfidf_glove(object):
    def __init__(self, embeddings_dict):
        # Glove Embeddings Dictionary
        self.embeddings_dict = embeddings_dict
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(embeddings_dict[next(iter(embeddings_dict))])
        # Initialize TF-IDF
        self.tfidf = TfidfVectorizer(min_df=3,
                                    token_pattern = '\S+', # preserve each word in preprocessed text
                                    max_df=0.9, 
                                    strip_accents='unicode',
                                    sublinear_tf=1)
        # TF-IDF weights dictionary
        self.weights_dict = None

    def fit(self, X, y):
        '''
        fit tfidf and create weight dictionary
        credit: github.com/nadbordrozd
        '''
        self.tfidf.fit(X)
        # if a word doesn't exist, it needs to be considered as 
        # infrequent as the most infrequent known words, i.e., 
        # it should have the highest known idf value.
        max_idf = max(self.tfidf.idf_)
        self.weights_dict = defaultdict(
            lambda: max_idf, 
            [(w, self.tfidf.idf_[i]) for w, i in self.tfidf.vocabulary_.items()])
        return self
    
    def sentence2vec(self, s):
        '''
        Input:
        Sentence string

        Transformations:
        Get vector for each word -> 
        weigh the vector by idf value ->
        Average vectors

        Output:
        Vector for sentence
        '''
        words = s.split()
        M = []
        for w in words:
            try:
                M.append(self.embeddings_dict[w] * self.weights_dict[w])
            except:
                continue
        M = np.array(M)
        v = M.mean(axis=0)
        if type(v) != np.ndarray:
            return np.zeros(self.dim)
        return v # / np.sqrt((v ** 2).sum())

    def transform(self, X):
        X = X.apply(self.sentence2vec)
        return np.stack(X.values, axis=0)

In [12]:
vec = tfidf_glove(embeddings_dict)
X_train = vec.fit(train['comment_text'], train[labels[0]]).transform(train['comment_text'])
X_valid = vec.transform(valid['comment_text'])

### Loop through Logistic Regression, SVM, and XGBoost

In [13]:
## CREATE RESULTS TABLE
results = pd.DataFrame(columns=['Label','Accuracy', 'Recall', 'Precision', 'F1', 'Vectorizer', 'model'])

## CREATE MODELS
models = {
    'Logistic Regression': LogisticRegression(solver='saga',class_weight='balanced'),
    'SVM': LinearSVC(class_weight='balanced'),
    'XGBoost': ['see below']}
                      
## LOOP THROUGH MODELS
for m_label, model in models.items():
    for label in labels:
        # clone base model (re-initialize weights)
        if m_label == 'XGBoost':
            model = XGBClassifier(n_estimators=100,
                        scale_pos_weight= sum(train[label]==0) / sum(train[label]==1),
                        n_jobs=-1)
        
        m = clone(model) 

        # Fit model
        m.fit(X_train, train[label])
        
        # Get predictions
        preds = m.predict(X_valid)

        # Evaluate predictions
        acc, prec, recall, f1 = (accuracy_score(valid[label], preds), 
                                precision_score(valid[label], preds), 
                                recall_score(valid[label], preds), 
                                f1_score(valid[label], preds))
        
        # Save results to dataframe
        results = results.append({'Label': label,
                                'Accuracy':acc,
                                'Recall':recall,
                                'Precision':prec,
                                'F1':f1,
                                'Vectorizer':'tfidf_glove',
                                'model': m_label}, 
                                ignore_index = True)
        
        # print results
        print('{0} Results for {1} comments: Accuracy - {2:.2f}; Precision - {3:.2f}; Recall - {4:.2f}; F1 - {5:.2f}'.format(
                                        m_label,
                                        label, 
                                        acc, 
                                        prec, 
                                        recall,
                                        f1))

Logistic Regression Results for toxic comments: Accuracy - 0.89; Precision - 0.46; Recall - 0.85; F1 - 0.60
Logistic Regression Results for severe_toxic comments: Accuracy - 0.95; Precision - 0.16; Recall - 0.89; F1 - 0.27
Logistic Regression Results for obscene comments: Accuracy - 0.92; Precision - 0.40; Recall - 0.86; F1 - 0.54
Logistic Regression Results for threat comments: Accuracy - 0.94; Precision - 0.04; Recall - 0.82; F1 - 0.07
Logistic Regression Results for insult comments: Accuracy - 0.91; Precision - 0.35; Recall - 0.86; F1 - 0.49
Logistic Regression Results for identity_hate comments: Accuracy - 0.92; Precision - 0.09; Recall - 0.88; F1 - 0.17
SVM Results for toxic comments: Accuracy - 0.93; Precision - 0.61; Recall - 0.72; F1 - 0.66
SVM Results for severe_toxic comments: Accuracy - 0.98; Precision - 0.33; Recall - 0.56; F1 - 0.41
SVM Results for obscene comments: Accuracy - 0.95; Precision - 0.52; Recall - 0.77; F1 - 0.62
SVM Results for threat comments: Accuracy - 0.99

In [14]:
## SAVE RESULTS
results.to_csv('../artifacts/tfidf_glove.csv', index=False)